In [9]:
!pip install tensorflow
!pip install matplotlib 
!pip install numpy scikit-learn pandas opencv-python 

  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 8.6 MB/s  0:00:00m eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import cv2
import os

In [11]:
#load images and do some preprocessing

#size for all images
image_resize = (128, 128)

#load and save the images 
def process_and_save(input_path, output_path):
    """
    Load an image, convert to grayscale, resize, and save.
    """
    img = cv2.imread(input_path)

    #for missing images 
    if img is None:
        print("Could not load:", input_path)
        return
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #covert to grayscale
    img = cv2.resize(img, image_resize) #resize
    cv2.imwrite(output_path, img) #save 


#save output path to a folder
def process_folder(input_dir, output_dir):
    """
    Process every image in an input folder and save to output.
    """
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        in_path = os.path.join(input_dir, filename)
        out_path = os.path.join(output_dir, filename)

        if os.path.isfile(in_path):
            process_and_save(in_path, out_path)

#folders for eyes to save the datasets
eye_out_open = "Processed_Datasets/eyes/open_eye"
eye_out_closed = "Processed_Datasets/eyes/closed_eye"

#folders containing raw data for eyes
eye_sources_open = ["Raw_Data/Open_Eyes"]
eye_sources_closed = ["Raw_Data/Closed_Eyes"]

#process open eys 
for fldr in eye_sources_open:
    process_folder(fldr, eye_out_open)

# PROCESS CLOSED EYES
for fldr in eye_sources_closed:
    process_folder(fldr, eye_out_closed)

#folders for mouths to save the datasets
mouth_out_yawn = "Processed_Datasets/mouth/yawn"
mouth_out_noyawn  = "Processed_Datasets/mouth/no_yawn"

#folders containing raw data for mouths
mouth_sources_yawn = ["Raw_Data/Yawn"]
mouth_sources_noyawn = ["Raw_Data/No_Yawn"]


# PROCESS YAWN MOUTHS
for fldr in mouth_sources_yawn:
    process_folder(fldr, mouth_out_yawn)

# PROCESS NO-YAWN MOUTHS
for fldr in mouth_sources_noyawn:
    process_folder(fldr, mouth_out_noyawn)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score   

**EYE CNN MODEL**

In [ ]:
#loading the processed datasets and preparing them for model training

def load_eye_dataset():
    X = []
    y = []

    # 0 = open, 1 = closed
    base_paths = {0: "Processed_Datasets/eyes/open_eye",
                  1: "Processed_Datasets/eyes/closed_eye"}

    for label, folder in base_paths.items():
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)

            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is None: #handles missing data
                continue

            img = img.astype("float32") / 255.0
            img = np.expand_dims(img, axis=-1) #Axis is 1 for grayscale

            X.append(img)
            y.append(label)

    return np.array(X), np.array(y)

X_eye, y_eye = load_eye_dataset()



In [15]:
#train test split
X_eye_train, X_eye_test, y_eye_train, y_eye_test = train_test_split(
    X_eye, y_eye,
    test_size=0.2, #20% test size
    stratify=y_eye, #equal class distribution
    random_state=42 #for reproducibility
)

#model architecture for eye state classification
def build_eye_cnn():
    model = models.Sequential([

        # Input shape: 128x128 grayscale
        layers.Input(shape=(128, 128, 1)),

        # Block 1
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),

        # Block 2
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),

        # Block 3
        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),

        # Dense layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),     # Prevent overfitting
        layers.Dense(1, activation='sigmoid') #output layer
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    return model

eye_model = build_eye_cnn()

history_eye = eye_model.fit(
    X_eye_train, y_eye_train,
    validation_data=(X_eye_test, y_eye_test),
    epochs=12,
    batch_size=32)

Epoch 1/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 300ms/step - accuracy: 0.5000 - loss: 0.7083 - val_accuracy: 0.5000 - val_loss: 0.6660
Epoch 2/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 347ms/step - accuracy: 0.6375 - loss: 0.5758 - val_accuracy: 0.9438 - val_loss: 0.3828
Epoch 3/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 304ms/step - accuracy: 0.9047 - loss: 0.2675 - val_accuracy: 0.9875 - val_loss: 0.1306
Epoch 4/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 298ms/step - accuracy: 0.9812 - loss: 0.1030 - val_accuracy: 0.9937 - val_loss: 0.0413
Epoch 5/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 303ms/step - accuracy: 0.9891 - loss: 0.0311 - val_accuracy: 1.0000 - val_loss: 0.0061
Epoch 6/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 405ms/step - accuracy: 0.9937 - loss: 0.0239 - val_accuracy: 1.0000 - val_loss: 0.0061
Epoch 7/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 440ms/step - accuracy: 1.0000 - loss: 0.0048 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 8/12
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 369ms/step - accuracy: 0.9969 - loss: 0.0090 - val_accuracy: 1.